In [ ]:
import os

from dotenv import load_dotenv

In [2]:
# set env vars
load_dotenv("./.env")
os.environ["LANGSMITH_PROJECT"] = "tool_calling_extraction"

In [ ]:
# define schema
from typing import Optional

from pydantic import BaseModel, Field


class Computer(BaseModel):
    """Specifications of any computing device"""

    model: Optional[str] = Field(
        description="The specific model number or name of the computing device."
    )
    storage: Optional[int] = Field(
        description="How many GBs the computng device has as the total storage."
    )
    num_cpu: Optional[int] = Field(
        description="The number of physical CPUs the device has.", ge=1
    )
    gpus: Optional[list[str]] = Field(
        description="The graphics processing units the comuting device has."
    )

In [ ]:
# design prompt
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert information extraction machine. Only extract relevant, accurat and true information from the input. If you don't know the value of an attribute you are asked to extract, you can return None. Don't make up information you don't know.",
        ),
        ("human", "{text}"),
    ]
)

In [6]:
# set up structured output
from langchain.chat_models import init_chat_model

llm = init_chat_model(model="gpt-5", model_provider="openai")

structured_llm = llm.with_structured_output(schema=Computer)

In [ ]:
# run inference
text = "I'm looking to upgrade my intel dual core proecssor. It's nvidia gpu is getting bottlenecked by the weak igpu. It's an old Lenovo thinkstation."
prompt = prompt_template.invoke(input={"text": text})
structured_llm.invoke(prompt)

Computer(model='Lenovo ThinkStation', storage=None, num_cpu=None, gpus=['NVIDIA GPU', 'Intel iGPU'])

In [ ]:
# extracting data from multiple entities
class Data(BaseModel):
    """Extracted data about computers"""

    computers: list[Computer]

In [8]:
# run inference for multiple entities
multiple_computer_text = """At the tech expo, several new computing devices were showcased.

First, the Acer Predator Helios 300 was on display, boasting 1TB of SSD storage, 1 CPU, and an NVIDIA GeForce RTX 3070 GPU.

Next was the Apple MacBook Pro M2, which includes 512GB of storage and the Apple M2 GPU, but details about the number of CPUs weren't provided.

Dell presented a workstation, model Dell Precision 7865, equipped with 2 CPUs, 2TB of storage, and two GPUs: NVIDIA RTX A6000 and Intel UHD Graphics 770.

A budget laptop labeled Lenovo IdeaPad 3 came with 256GB storage and 1 CPU, but no GPU was listed.

Finally, there was a compact mini-PC, simply called ZBox QX, which runs with 1 CPU, 1TB of storage, and an NVIDIA Quadro P5000 GPU."""

multiple_entity_prompt = prompt_template.invoke(input={"text": multiple_computer_text})
multiple_entity_structured_llm = llm.with_structured_output(schema=Data)
multiple_entity_structured_llm.invoke(multiple_entity_prompt)

Data(computers=[Computer(model='Acer Predator Helios 300', storage=1000, num_cpu=1, gpus=['NVIDIA GeForce RTX 3070']), Computer(model='Apple MacBook Pro M2', storage=512, num_cpu=None, gpus=['Apple M2 GPU']), Computer(model='Dell Precision 7865', storage=2000, num_cpu=2, gpus=['NVIDIA RTX A6000', 'Intel UHD Graphics 770']), Computer(model='Lenovo IdeaPad 3', storage=256, num_cpu=1, gpus=None), Computer(model='ZBox QX', storage=1000, num_cpu=1, gpus=['NVIDIA Quadro P5000'])])

In [9]:
# few shot example

mesages = [
    ("human", "1 🏀 2"),
    ("ai", "2"),
    ("human", "3 🏀 4"),
    ("ai", "12"),
    ("human", "5 🏀 6"),
]
llm.invoke(mesages)

AIMessage(content='30', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 266, 'prompt_tokens': 46, 'total_tokens': 312, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 256, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-5-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-C4OXQiHqO9VwWMNfNqmh2FuvSIMsj', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--dcb4c7d3-4caa-46d5-835b-96cd12872c87-0', usage_metadata={'input_tokens': 46, 'output_tokens': 266, 'total_tokens': 312, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 256}})

- Structured output uses tool calling at a low level, using the Pydantic schema to verify the output
- Some model providers require an ai message after the tool call
- The `tool_example_to_messages` will do this for us according to the model provider

In [10]:
from langchain_core.messages import BaseMessage
from langchain_core.utils.function_calling import tool_example_to_messages

examples = [
    (
        "The Bajaj Pulsar N160 boasts dual abs breaks and a 160cc engine, all complimented by the telescopic suspension on the front and the back.",
        Data(computers=[]),
    ),
    (
        "I'm looking to upgrade my intel dual core proecssor. It's nvidia gpu is getting bottlenecked by the weak igpu. It's an old Lenovo thinkstation.",
        Data(
            computers=[
                Computer(
                    model="Lenovo ThinkStation",
                    storage=None,
                    num_cpu=1,
                    gpus=["NVIDIA GPU"],
                )
            ]
        ),
    ),
]

messages: list[BaseMessage] = []

for input_text, tool_call in examples:
    if tool_call.computers:
        ai_message = "Computer info extracted!"
    else:
        ai_message = "No computers detected!"
    messages.extend(
        tool_example_to_messages(
            input=input_text, tool_calls=[tool_call], ai_response=ai_message
        )
    )

/tmp/ipykernel_150106/1444525344.py:32: LangChainBetaWarning: The function `tool_example_to_messages` is in beta. It is actively being worked on, so the API may change.
  tool_example_to_messages(


In [11]:
for i in messages:
    i.pretty_print()

================================ Human Message =================================

The Bajaj Pulsar N160 boasts dual abs breaks and a 160cc engine, all complimented by the telescopic suspension on the front and the back.
================================== Ai Message ==================================
Tool Calls:
  Data (7b856892-e79d-40a1-a5cf-13de82c23f2a)
 Call ID: 7b856892-e79d-40a1-a5cf-13de82c23f2a
  Args:
    computers: []
================================= Tool Message =================================

You have correctly called this tool.
================================== Ai Message ==================================

No computers detected!
================================ Human Message =================================

I'm looking to upgrade my intel dual core proecssor. It's nvidia gpu is getting bottlenecked by the weak igpu. It's an old Lenovo thinkstation.
================================== Ai Message ==================================
Tool Calls:
  Data (fe46f225-2f9b-4f7

In [12]:
# Create prompt template with few-shot examples
base_messages = [
    (
        "system",
        "You are an expert information extraction machine. Only extract relevant, accurat and true information from the input. If you don't know the value of an attribute you are asked to extract, you can return None. Don't make up information you don't know.",
    )
]

# Add the few-shot examples
base_messages.extend(messages)

# Add the final human input
base_messages.append(("human", "{text}"))

# Create the new prompt template
few_shot_prompt_template = ChatPromptTemplate.from_messages(base_messages)

In [ ]:
# Test the few-shot prompt template
test_text = (
    "I have a Dell XPS 15 with 16GB RAM, 512GB SSD, and NVIDIA GTX 1650 graphics card."
)

system_message = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert information extraction machine. Only extract relevant, accurat and true information from the input. If you don't know the value of an attribute you are asked to extract, you can return None. Don't make up information you don't know.",
        )
    ]
)


few_shot_prompt_template = ChatPromptTemplate.from_messages(
    [system_message + messages + [("human", "{input}")]]
)
few_shot_prompt_template = few_shot_prompt_template.invoke(input={"input": test_text})
multiple_entity_structured_llm.invoke(few_shot_prompt_template)

Data(computers=[Computer(model='Dell XPS 15', storage=512, num_cpu=None, gpus=['NVIDIA GTX 1650'])])

In [ ]:
from pprint import pp

for i in few_shot_prompt_template.__dict__.get("messages"):
    i.pretty_print()

================================ System Message ================================

You are an expert information extraction machine. Only extract relevant, accurat and true information from the input. If you don't know the value of an attribute you are asked to extract, you can return None. Don't make up information you don't know.
================================ Human Message =================================

The Bajaj Pulsar N160 boasts dual abs breaks and a 160cc engine, all complimented by the telescopic suspension on the front and the back.
================================== Ai Message ==================================
Tool Calls:
  Data (7b856892-e79d-40a1-a5cf-13de82c23f2a)
 Call ID: 7b856892-e79d-40a1-a5cf-13de82c23f2a
  Args:
    computers: []
================================= Tool Message =================================

You have correctly called this tool.
================================== Ai Message ==================================

No computers detected!
===========